<a href="https://colab.research.google.com/github/MehrdadJalali-AI/GraphMOF-AI/blob/main/MOFGalaxyNet_LLM_Neo4j.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install neo4j torch torch_geometric networkx openai

In [ ]:
import torch
import networkx as nx
import openai
from neo4j import GraphDatabase
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import numpy as np

In [ ]:
# Connect to Neo4j (Replace with your credentials)
NEO4J_URI = "bolt://localhost:7687"
USERNAME = "neo4j"
PASSWORD = "your_password"
driver = GraphDatabase.driver(NEO4J_URI, auth=(USERNAME, PASSWORD))

# Function to create MOF Graph
def create_mof_graph(tx):
    mof_data = [
        ("MOF-5", 0.8, 0.9),
        ("UiO-66", 0.75, 0.95),
        ("HKUST-1", 0.85, 0.8),
        ("MIL-101", 0.9, 0.88),
        ("ZIF-8", 0.7, 0.92)
    ]

    for mof, adsorption, stability in mof_data:
        tx.run("""
        MERGE (m:MOF {name: $name})
        SET m.adsorption = $adsorption, m.stability = $stability
        """, name=mof, adsorption=adsorption, stability=stability)

    edges = [("MOF-5", "UiO-66"), ("MOF-5", "HKUST-1"), ("HKUST-1", "MIL-101"), ("UiO-66", "ZIF-8")]
    for mof1, mof2 in edges:
        tx.run("""
        MATCH (m1:MOF {name: $mof1}), (m2:MOF {name: $mof2})
        MERGE (m1)-[:SIMILAR_TO]->(m2)
        """, mof1=mof1, mof2=mof2)

with driver.session() as session:
    session.write_transaction(create_mof_graph)
print("✅ MOF Graph Created in Neo4j!")

In [ ]:
def create_mof_networkx():
    G = nx.Graph()
    mof_nodes = ["MOF-5", "UiO-66", "HKUST-1", "MIL-101", "ZIF-8"]
    for node in mof_nodes:
        G.add_node(node, adsorption=torch.rand(1).item(), stability=torch.rand(1).item())
    edges = [("MOF-5", "UiO-66"), ("MOF-5", "HKUST-1"), ("HKUST-1", "MIL-101"), ("UiO-66", "ZIF-8")]
    G.add_edges_from(edges)
    return G

def graph_to_pyg(G):
    node_mapping = {node: i for i, node in enumerate(G.nodes)}
    edge_index = torch.tensor([[node_mapping[u], node_mapping[v]] for u, v in G.edges], dtype=torch.long).t().contiguous()
    features = torch.tensor([[G.nodes[node]["adsorption"], G.nodes[node]["stability"]] for node in G.nodes], dtype=torch.float)
    return Data(x=features, edge_index=edge_index)

In [ ]:
class MOFGraphGNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(MOFGraphGNN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

def train_gnn(data):
    model = MOFGraphGNN(in_channels=2, hidden_channels=16, out_channels=4)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    loss_fn = torch.nn.MSELoss()
    for epoch in range(200):
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = loss_fn(out, data.x)
        loss.backward()
        optimizer.step()
    return model, out

G = create_mof_networkx()
data = graph_to_pyg(G)
model, embeddings = train_gnn(data)
print("✅ GNN Training Complete! Extracted Embeddings:", embeddings.detach().numpy())

In [ ]:
openai.api_key = "YOUR_OPENAI_API_KEY"

def llm_mof_insights(embeddings, mof_names):
    insights = {}
    for i, mof in enumerate(mof_names):
        embedding_str = ", ".join([f"{v:.4f}" for v in embeddings[i].tolist()])
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[{"role": "user", "content": f"Insights on {mof} with embedding {embedding_str}"}]
        )
        insights[mof] = response["choices"][0]["message"]["content"]
    return insights

mof_names = list(G.nodes)
insights = llm_mof_insights(embeddings.detach().numpy(), mof_names)
print(insights)